# Trash Rotation Tracker Generator

Use this notebook to generate a custom printable trash rotation tracker.

## Instructions
1. Edit the names in the configuration cell below.
2. Run all cells (Cell > Run All).
3. Find the generated Excel file in the `output` folder.

In [ ]:
# --- Configuration ---
# Enter the names of the people in the rotation order.
names = ["KM", "NP", "PS", "LS"] 

# Number of rows (weeks/turns) to generate
rows = 50

# Output filename
output_filename = "Trash_Rotation_Printable_Custom.xlsx"
# ---------------------

In [ ]:
import pandas as pd
import os

# 1. Setup Data
dent_bracket = "[     ]"
data = {"Turn #": list(range(1, rows + 1))}
for name in names:
    data[name] = [dent_bracket] * rows

df = pd.DataFrame(data)

# Add Totals Row
totals = ["TOTALS"] + ["____"] * len(names)
df.loc[rows] = totals

# 2. Create Excel File
output_dir = "../output"
os.makedirs(output_dir, exist_ok=True)
filepath = os.path.join(output_dir, output_filename)

writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1', index=False, startrow=3, header=False)

workbook = writer.book
worksheet = writer.sheets['Sheet1']
worksheet.hide_gridlines(2)

# 3. Define Styles
base_props = {'font_name': 'Courier New', 'font_size': 11, 'align': 'center', 'valign': 'vcenter'}

format_left = workbook.add_format({**base_props, 'left': 2, 'right': 1, 'top': 1, 'bottom': 1})
format_mid  = workbook.add_format({**base_props, 'left': 1, 'right': 1, 'top': 1, 'bottom': 1})
format_right = workbook.add_format({**base_props, 'left': 1, 'right': 2, 'top': 1, 'bottom': 1})

format_total_left = workbook.add_format({**base_props, 'bold': True, 'font_size': 12, 'left': 2, 'right': 1, 'top': 1, 'bottom': 2})
format_total_mid = workbook.add_format({**base_props, 'bold': True, 'font_size': 12, 'left': 1, 'right': 1, 'top': 1, 'bottom': 2})
format_total_right = workbook.add_format({**base_props, 'bold': True, 'font_size': 12, 'left': 1, 'right': 2, 'top': 1, 'bottom': 2})

format_header_base = {**base_props, 'bold': True, 'fg_color': '#D7E4BC', 'font_size': 14, 'top': 2, 'bottom': 1}

title_format = workbook.add_format({'bold': True, 'font_size': 18, 'align': 'center', 'valign': 'vcenter', 'font_name': 'Arial', 'text_wrap': True})
instr_format = workbook.add_format({'italic': True, 'font_size': 10, 'align': 'center', 'valign': 'vcenter', 'font_name': 'Arial', 'text_wrap': True})

# 4. Apply Formatting
# Column Widths
worksheet.set_column(0, 0, 10) # Col A
worksheet.set_column(1, len(names), 22) # Cols B onwards

# Header
headers = df.columns.values
for col, value in enumerate(headers):
    p = format_header_base.copy()
    if col == 0:
        p['left'] = 2; p['right'] = 1
    elif col == len(headers) - 1:
        p['left'] = 1; p['right'] = 2
    else:
        p['left'] = 1; p['right'] = 1
    f = workbook.add_format(p)
    worksheet.write(3, col, value, f)

# Data Rows
start_data_row = 4
for r in range(rows):
    current_excel_row = start_data_row + r
    # Turn number
    worksheet.write(current_excel_row, 0, data["Turn #"][r], format_left)
    # Names
    for i in range(len(names)):
        col_idx = i + 1
        style = format_right if col_idx == len(headers) - 1 else format_mid
        worksheet.write(current_excel_row, col_idx, dent_bracket, style)

# Totals Row
totals_row = start_data_row + rows
worksheet.write(totals_row, 0, "TOTALS", format_total_left)
for i in range(len(names)):
    col_idx = i + 1
    style = format_total_right if col_idx == len(headers) - 1 else format_total_mid
    worksheet.write(totals_row, col_idx, "____", style)

# Title & Instructions
last_col_idx = len(names)
last_col_letter = chr(65 + last_col_idx)
title_range = f'A1:{last_col_letter}1'
instr_range = f'A2:{last_col_letter}2'

worksheet.set_row(0, 30)
worksheet.set_row(1, 20)
worksheet.set_row(3, 25)

worksheet.merge_range(title_range, 'TRASH ROTATION TRACKER', title_format)

flow_text = "FLOW: " + " -> ".join(names) + " -> (Loop)"
worksheet.merge_range(instr_range, flow_text, instr_format)

footer_row = totals_row + 2
worksheet.set_row(footer_row, 30)
footer_range = f'A{footer_row+1}:{last_col_letter}{footer_row+1}'
worksheet.merge_range(footer_range, "INSTRUCTION: Find first empty box. Press House Key firmly into box [     ] to mark done.", instr_format)

# Page Layout
worksheet.set_paper(9) # A4
worksheet.fit_to_pages(1, 1)
worksheet.center_horizontally()
worksheet.set_margins(left=0.5, right=0.5, top=0.5, bottom=0.5)
worksheet.print_area(0, 0, footer_row+1, len(names))

writer.close()
print(f"Success! File '{filepath}' created.")